In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline

In [5]:
from sklearn.utils import shuffle

In [9]:
import Io_tf_binary_general as io

In [29]:
#utworze teraz dane jakies
def kolko_w_kolku(klasa,sig1=0.4,sig2=0.6,R2=1.,N=1000):
    """to ma utworzyc dane ktore posluza mi do sprawdzenia czy i jak dziala model
    klasa to 0 lub 1"""
    
    if klasa==1:
        x=np.random.normal(0,sig1)
        y=np.random.normal(0,sig1)
        #features.append([x,y])
        return {"pierwsza":x,"inna_wlasnosc":y}
    else:
        alpha=np.random.uniform()*2*np.pi
        delr=np.random.normal(0,sig2)
        r=R2+delr
        x=np.cos(alpha)*r
        y=np.sin(alpha)*r
        #features.append([x,y])
        return {"pierwsza":x,"inna_wlasnosc":y}

In [35]:
p0_train=0.3
p1_train=0.7
p0_validate=0.2
p1_validate=0.8
#to pierwse to prawdopodobienstwo, ze cos z pierwszej populacji jest naprawdę 1
# to drugie to prawdopodobiensto, ze cos z drugiej populacji jest naprawde 1

In [45]:
pisacz=io.Io_tf_binary_general("weak_train",'w')
for i in range(10000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=int(np.random.uniform()<p0_train)
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=int(np.random.uniform()<p1_train)
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [46]:
pisacz=io.Io_tf_binary_general("weak_validate",'w')
for i in range(1000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=int(np.random.uniform()<p0_validate)
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=int(np.random.uniform()<p1_validate)
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [48]:
pisacz=io.Io_tf_binary_general("weak_true",'w')
for i in range(1000):
    klasa=np.random.randint(0,2)
    if klasa==0:
        prawdziwa=klasa
        pisacz.write_old(kolko_w_kolku(prawdziwa),0)
    else:
        prawdziwa=klasa
        pisacz.write_old(kolko_w_kolku(prawdziwa),1)
pisacz.close()

In [47]:
czytacz=io.Io_tf_binary_general("weak_validate",'r')
dataset=czytacz.read()
BATCH_SIZE=2
zbachowany=dataset.shuffle(1000).repeat().batch(BATCH_SIZE)
iterator = zbachowany.make_one_shot_iterator()
para=iterator.get_next()

with tf.Session() as sess:
    for i in range(3):
        print(sess.run(para))

{'pierwsza': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'inna_wlasnosc': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'label': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}
({'inna_wlasnosc': array([[0.8199839],
       [0.6351721]], dtype=float32), 'pierwsza': array([[0.2276179],
       [1.0212047]], dtype=float32)}, array([0, 0]))
({'inna_wlasnosc': array([[-0.21452516],
       [-0.08526361]], dtype=float32), 'pierwsza': array([[0.05460102],
       [0.10392821]], dtype=float32)}, array([0, 1]))
({'inna_wlasnosc': array([[-0.62881684],
       [-0.10491646]], dtype=float32), 'pierwsza': array([[ 0.79364765],
       [-0.51056725]], dtype=float32)}, array([0, 1]))
